In [1]:
%jsroot on

In [2]:
#include<TFile.h>
#include<TH1.h>
#include<TLine.h>
#include<iostream>
#include<string>
#include<stdio.h>

In [3]:
TFile *f=new TFile("//Analysis/Output_HZZAnalysis/data.root");
//TFile *f = new TFile("//Analysis/Output_HZZAnalysis/ggH125_ZZ4lep.root"); // Prueba MC gg->H125->ZZ->4lep.
mass_four_lep->Print(); // Test de lectura de datos

TH1.Print Name  = mass_four_lep, Entries= 321, Total sum= 95


In [4]:
// gROOT->SetStyle("ATLAS"); // Establecer estilo.
gStyle->SetOptStat("emr"); // Dibujar estadisticas de grafico.
gStyle->SetOptFit(1112);

In [5]:
TCanvas *c1 = new TCanvas("c1","c1",10,10,900,600);

In [6]:
TH1F *m_four_lep = (TH1F*)f->Get("mass_four_lep");
m_four_lep->Draw();
//m_four_lep->Draw("E1"); // E1: Para representar los puntos con barras de error.

In [7]:
Int_t fit = 0; // Option 0, 1, 2, 3 = Distribuciones

In [8]:
Double_t mybw(Double_t* x, Double_t* par) // Distribución de Breit-Wigner relativista
{
    Double_t arg1 = 2/TMath::Pi();
    Double_t arg2 = pow(par[1],2)*pow(par[2],2); // Gamma=par[1] M=par[2]
    Double_t arg3 = pow((pow(x[0],2)-pow(par[2],2)),2);
    Double_t arg4 = pow(x[0],4)*(pow(par[1],2)/pow(par[2],2));
    return par[0]*arg1*arg2/(arg3+arg4);
}

In [9]:
if (fit == 0) {
    TF1 *f1 = new TF1("f1","mybw",108,140,3);
    f1->SetParameters(1,m_four_lep->GetMean(),m_four_lep->GetRMS());
    //f1->SetParNames ("Constant","Mean","Sigma"); // OJO con el tipo de parámetro.
}
if (fit == 1) {TF1 *f1 = new TF1("f1","gaus",110,140);}
if (fit == 2) {TF1 *f1 = new TF1("f1","landau",110,140);}
if (fit == 3) {TF1 *f1 = new TF1("f1","gausn",110,140);}

In [10]:
m_four_lep->Fit("f1","R"); // Puede usarse V para agregar más detalles estadísticos.
f1->SetLineColor(kRed);

 FCN=3.3306 FROM MIGRAD    STATUS=CONVERGED     224 CALLS         225 TOTAL
                     EDM=1.10162e-06    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.52306e+01   4.71651e+00   2.83250e-03  -5.21077e-04
   2  p1           8.92687e+00   2.78212e+00   1.64098e-03  -7.67702e-04
   3  p2           1.24732e+02   1.14059e+00   1.08552e-03  -1.66526e-04


In [11]:
// Print the full information of the fit including covariance matrix.
f1->Print("V"); 

// Store the result in a ROOT file.
//f1->Write(); 

Formula based function:     f1 
            tf1lambda : mybw Ndim= 1, Npar= 3, Number= 0 
 Formula expression: 
	mybw 
List of  Parameters: 
Par   0                   p0 =   15.230633 
Par   1                   p1 =    8.926867 
Par   2                   p2 =  124.731604 
Expression passed to Cling:
	mybw


In [12]:
minx = f1->GetXmin(); // Mínimo en x del ajuste.
maxx = f1->GetXmax(); // Máximo en x del ajuste.

mode_x = f1->GetMaximumX(minx,maxx); // Coordenada x del máximo en el ajuste dentro del intervalo [minx,maxx].
mode_y = f1->Eval(mode_x); // Coordenada y del máximo.

upper_halfwidth = f1->GetX(mode_y/2.0,mode_x,maxx);
lower_halfwidth = f1->GetX(mode_y/2.0,minx,mode_x);

width = upper_halfwidth-lower_halfwidth;

cout    << "Coordenada del punto máximo en el ajuste: (" << mode_x << "," << mode_y << ")." << endl
        << "Intersecciones en m4l a média altura: " << lower_halfwidth << " GeV" << " y " << upper_halfwidth << " GeV." << endl
        << "Ancho de decaimiento, \u0393 = " << width << " GeV." << endl
        << "Tiempo de vida media, \u03C4 = " << 4.135667696E-15/(width*1E9) << " s." << endl;

TLine *l1 = new TLine(lower_halfwidth,mode_y/2.0,upper_halfwidth,mode_y/2.0);
l1->SetLineColor(kGreen);
l1->SetLineStyle(7);
l1->Draw("same");

TLine *l2 = new TLine(mode_x,mode_y,mode_x,0);
l2->SetLineColor(kGreen);
l2->SetLineStyle(7);
l2->Draw("same");

c1->Draw();

Coordenada del punto máximo en el ajuste: (124.413,9.74579).
Intersecciones en m4l a média altura: 119.879 GeV y 128.789 GeV.
Ancho de decaimiento, Γ = 8.90981 GeV.
Tiempo de vida media, τ = 4.6417e-25 s.


#### OJO Revisar los orden de magnitud y unidades.